In [ ]:
import os

# set the current working directory
curr_path = os.getcwd().split("/")[:-1]
curr_path = "/".join(curr_path)
os.chdir(curr_path)

In [1]:
import numpy as np
# read dataset
from sclibrary import load_dataset


sc, _, flow = load_dataset()"forex")
# get the y from flow dict
y = np.fromiter(flow.values(), dtype=float)

print("Flow:", len(y))

Num. of nodes: 25
Num. of edges: 210
Num. of triangles: 770
Shape: (25, 210, 770)
Max Dimension: 2
Generating coordinates using spring layout.
Flow: 210


In [2]:
from sclibrary.hogde_gp import HodgeGPTrainer

B1 = sc.incidence_matrix(rank=1).toarray()
B2 = sc.incidence_matrix(rank=2).toarray()
L1 = sc.hodge_laplacian_matrix(rank=1).toarray()
L1l = sc.lower_laplacian_matrix(rank=1).toarray()
L1u = sc.upper_laplacian_matrix(rank=1).toarray()

hogde_gp = HodgeGPTrainer(
    B1 = B1,
    B2 = B2,
    L1 = L1,
    L1l = L1l,
    L1u = L1u,
    y = y,
)

In [3]:
train_ratio = 0.2
data_normalization = False

laplacians = hogde_gp.get_laplacians()
incidence_matrices = hogde_gp.get_incidence_matrices()
eigenpairs = hogde_gp.get_eigenpairs()

x_train, y_train, x_test, y_test, x, y  = hogde_gp.train_test_split(train_ratio=train_ratio, data_normalization=data_normalization)

L1:  (210, 210)
L1l:  (210, 210)
L1u:  (210, 210)
B1:  (25, 210)
B2:  (210, 770)
x_train: (42,)
x_test: (168,)
y_train: (42,)
y_test: (168,)


In [7]:
x_train[0], y_train[0]

(tensor(34.), tensor(4.4740))

In [17]:
from sclibrary.hogde_gp.kernel_serializer import KernelSerializer

kernel_type = "matern"
data_name = "forex"

kernel = KernelSerializer().serialize(eigenpairs=eigenpairs, kernel_type=kernel_type, data_name=data_name)

In [18]:
import gpytorch
from sclibrary.hogde_gp import ExactGPModel

likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood, kernel, mean_function=None)

In [19]:
type(kernel)

sclibrary.hogde_gp.forex.matern.MaternKernelForex

In [20]:
import torch
output_device = "cpu"

if torch.cuda.is_available():
   model = model.to(output_device)
   likelihood = likelihood.to(output_device)

In [21]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:50} value = {param.item()}')

Parameter name: likelihood.noise_covar.raw_noise                   value = 0.0
Parameter name: mean_module.raw_constant                           value = 0.0
Parameter name: covar_module.raw_outputscale                       value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa_down            value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa_up              value = 0.0
Parameter name: covar_module.base_kernel.raw_kappa                 value = 0.0
Parameter name: covar_module.base_kernel.raw_mu                    value = 0.0
Parameter name: covar_module.base_kernel.raw_mu_down               value = 0.0
Parameter name: covar_module.base_kernel.raw_mu_up                 value = 0.0
Parameter name: covar_module.base_kernel.raw_h                     value = 0.0
Parameter name: covar_module.base_kernel.raw_h_down                value = 0.0
Parameter name: covar_module.base_kernel.raw_h_up                  value = 0.0


In [22]:
model.train()
likelihood.train()
hogde_gp.train(model, likelihood, x_train, y_train)

Iter 1/1000 - Loss: 3.867 
Iter 2/1000 - Loss: 3.571 
Iter 3/1000 - Loss: 3.321 
Iter 4/1000 - Loss: 3.112 
Iter 5/1000 - Loss: 2.939 
Iter 6/1000 - Loss: 2.795 
Iter 7/1000 - Loss: 2.676 
Iter 8/1000 - Loss: 2.577 
Iter 9/1000 - Loss: 2.495 
Iter 10/1000 - Loss: 2.426 
Iter 11/1000 - Loss: 2.369 
Iter 12/1000 - Loss: 2.322 
Iter 13/1000 - Loss: 2.283 
Iter 14/1000 - Loss: 2.251 
Iter 15/1000 - Loss: 2.226 
Iter 16/1000 - Loss: 2.206 
Iter 17/1000 - Loss: 2.190 
Iter 18/1000 - Loss: 2.179 
Iter 19/1000 - Loss: 2.170 
Iter 20/1000 - Loss: 2.164 
Iter 21/1000 - Loss: 2.160 
Iter 22/1000 - Loss: 2.157 
Iter 23/1000 - Loss: 2.156 
Iter 24/1000 - Loss: 2.155 
Iter 25/1000 - Loss: 2.155 
Iter 26/1000 - Loss: 2.155 
Iter 27/1000 - Loss: 2.155 
Iter 28/1000 - Loss: 2.155 
Iter 29/1000 - Loss: 2.155 
Iter 30/1000 - Loss: 2.155 
Iter 31/1000 - Loss: 2.154 
Iter 32/1000 - Loss: 2.153 
Iter 33/1000 - Loss: 2.152 
Iter 34/1000 - Loss: 2.151 
Iter 35/1000 - Loss: 2.149 
Iter 36/1000 - Loss: 2.147 
I

In [23]:
hogde_gp.predict(model, likelihood, x_test, y_test)

Test MAE: 0.011434121988713741
Test MSE: 0.001980280503630638
Test R2: 0.9996825456619263
Test MLSS: -3.012634754180908
Test NLPD: -3.5197606086730957


MultivariateNormal(loc: torch.Size([168]))